# Notebook dedicated to the Coursera Capstone project

In [1]:
import pandas as pd
import numpy as np

print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Segmenting and Clustering Neighborhoods in Toronto

In [10]:
import requests
import lxml.html as lh

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page = requests.get(url)
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

column_names = ['PostalCode', 'Borough', 'Neighborhood'] 
neighborhoods = pd.DataFrame(columns=column_names)


for index in range(1,len(tr_elements)):
    row = tr_elements[index]
    
    if "Canadian postal codes" not in row.text_content():
        cell_index = 0
        postcode = ""
        borough = ""
        neighborhood = ""
        for cell in row.iterchildren():
            content = cell.text_content()
            
            if cell_index == 0:
                postcode = content
                
            elif cell_index == 1:
                borough = content
                
            elif cell_index == 2:
                neighborhood = content
            cell_index += 1
            
        if "Not assigned" not in borough:
            if "Not assigned" in neighborhood:
                neighborhood = borough
            neighborhood = neighborhood.replace("\n","")
            
            neighborhoods = neighborhoods.append({'PostalCode': postcode, 'Borough': borough, 'Neighborhood': neighborhood}, ignore_index=True)
            
    else:
        break
    
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [11]:
neighborhoods.shape

(212, 3)

In [13]:
new = neighborhoods.groupby(['PostalCode', 'Borough'])['Neighborhood'].agg(lambda col: ', '.join(col))
new

PostalCode  Borough         
                                                                                 
M1B         Scarborough                                            Rouge, Malvern
M1C         Scarborough                    Highland Creek, Rouge Hill, Port Union
M1E         Scarborough                         Guildwood, Morningside, West Hill
M1G         Scarborough                                                    Woburn
M1H         Scarborough                                                 Cedarbrae
M1J         Scarborough                                       Scarborough Village
M1K         Scarborough               East Birchmount Park, Ionview, Kennedy Park
M1L         Scarborough                           Clairlea, Golden Mile, Oakridge
M1M         Scarborough           Cliffcrest, Cliffside, Scarborough Village West
M1N         Scarborough                               Birch Cliff, Cliffside West
M1P         Scarborough         Dorset Park, Scarborough Town Centre,

In [14]:
new.shape

(104,)